# Examining the raw data I downloaded, and doing some cleaning, summarizing etc.

## Some issues with data:
- Some temps are -99999 (bad data flag?)
- Some rows have max temps less than min temps! 

## Plan
- do one station at a time
- Check for and remove duplicate rows
- Remove rows where temps are bad, according to following conditions:
- temp values = -99999 : this is about 4.5% of rows in the database
- temp <-50 | >150
- max temp < min temp

In [1]:
# %load /Users/Andy/jupyter_imports.py
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# make plots look nice
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['xtick.labelsize'] = 'large'
plt.rcParams['ytick.labelsize'] = 'large'
plt.rcParams['lines.linewidth'] = 3

import sqlite3
con = sqlite3.connect('/Users/Andy/Projects/wunderground_data/wunderground_daily.db')

In [ ]:
# check for bad values (flagged as -99999)
dat = pd.read_sql_query("SELECT * from wea where mean_temp==-99999 or max_temp==-99999 or mean_temp==-9999 ",con)
#dat.head()

In [ ]:
nr = pd.read_sql_query("select count(*) from wea  ",con)
nr.values

In [ ]:
# what percent of data is bad?
nrb = dat.shape[0]
#nrb
nrb / nr.values * 100 

In [ ]:
# abnormally high temperatures not flagged as -99999 in raw data
dat = pd.read_sql_query("SELECT * FROM wea WHERE mean_temp>150 and mean_temp!=-99999",con)
dat.head()
#dat.shape


In [ ]:
# abnormally low temperatures not flagged as -99999 in raw data
dat = pd.read_sql_query("SELECT * FROM wea WHERE mean_temp<-100 and mean_temp!=-99999",con)
dat.head()
dat.shape

In [ ]:
dat.head()

In [ ]:
dat = pd.read_sql_query("select * from wea where st_code='KDEN' ",con)
#dat.loc[dat.mean_temp<-50]
dat.describe()

In [ ]:
dat = pd.read_sql_query("select * from wea where st_code='KSEA' ",con)
len1 = dat.shape[0]
dat.date = pd.to_datetime(dat.date)
dat.where(dat.mean_temp>-50,inplace=True)
dat.where(dat.mean_temp<150,inplace=True)
dat.drop_duplicates(inplace=True)
len2 = dat.shape[0]

In [ ]:
len1-len2

In [ ]:

#plt.ylim(30,80)

In [ ]:

#plt.ylim(30,80)

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(dat.date,dat.max_temp)
plt.grid();

In [ ]:
# some where max_temp < min temp!!!!!
dat[dat.max_temp==0]

## Now write script/function to remove all bad data 

In [ ]:
stcode = 'KSEA'
dat = pd.read_sql_query("select * from wea where st_code=? ",con,params=[stcode])
#dat.info()
dat.head()

In [ ]:
dat.info()

In [ ]:
dat['precip_In'][dat['precip_In']=='T']='0.01'
dat['precip_In']=pd.to_numeric(dat['precip_In'])
dat.head()

In [2]:

def clean_data(dat):

    dat.date = pd.to_datetime(dat.date)

    Tmin = -100
    Tmax =  150

    dat.where(dat.mean_temp>Tmin, inplace=True)
    dat.where(dat.mean_temp<Tmax, inplace=True)

    dat.where(dat.min_temp>Tmin, inplace=True)
    dat.where(dat.min_temp<Tmax, inplace=True)

    dat.where(dat.max_temp>Tmin, inplace=True)
    dat.where(dat.max_temp<Tmax, inplace=True)

    dat.where(dat.max_temp > dat.min_temp, inplace=True)

    dat.drop_duplicates(inplace=True)
    
    # convert 'Trace' precip to 0.01
    # I want to preserve the fact that there was rain, but very small amount
    # Also, sometimes precip_In is read in as float, not object? (maybe if it contains no 'T'?)
    if dat.precip_In.dtype=='object':
        dat['precip_In'][dat['precip_In']=='T'] = '0.01'
        dat.precip_In = pd.to_numeric(dat.precip_In)
    
    # some precip values are way too large
    # According to weather.com :The most extreme 24-hour rainfall total on record 
    # in the U.S. is 42.0 inches near Alvin, Texas, between 7 a.m. July 25 and 7 a.m.
    # July 26, 1979.
    # I will keep values < 50
    dat.where(dat.precip_In<50, inplace=True)
    
    # screen unreasonably high winds
    dat.where(dat.max_gust_mph<300, inplace=True)
    
    return dat


In [ ]:

dat = clean_data(dat)
dat

In [ ]:
dat.info()

In [ ]:
plt.figure(figsize=(18,8))
#plt.plot(dat.date,dat.min_temp)
plt.plot(dat.date,dat.mean_temp)
#plt.plot(dat.date,dat.max_temp)
plt.grid();
plt.xlabel('date')
plt.ylabel('mean temp')
plt.title(stcode)

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(dat.date, dat.precip_In,'.')
plt.grid();

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(dat.date, dat.max_gust_mph,'.')
plt.grid();

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(dat.date, dat.cloud_cover,'.')
plt.grid();

## Loop through all stations and make cleaned data
- save cleaned csv files, with all data for each station (not yearly files)
- make new database with cleaned data

In [3]:
sta_df = pd.read_csv('USAirportWeatherStations.csv')
sta_df.head()

,Unnamed: 0,Station,State,airportCode,Lat,Lon,Elevation,WMO
0,1,Central,AK,PARL,65.57,-144.80,292,99999
1,2,Atka,AK,PAAK,52.22,-174.20,17,99999
2,3,Buckland,AK,PABL,65.99,-161.12,0,99999
3,4,Portage Glacier,AK,PATO,60.79,-148.83,29,99999
4,5,Nivalina,AK,PAVL,67.73,-164.55,3,70148


In [4]:
st_list = sta_df['airportCode'].values
st_list

array(['PARL', 'PAAK', 'PABL', ..., 'PGRO', 'PGSN', 'PGWT'], dtype=object)

In [5]:
st_list[0]

'PARL'

In [6]:
sta_df[sta_df.airportCode=='KSZY']

,Unnamed: 0,Station,State,airportCode,Lat,Lon,Elevation,WMO
1281,1282,Selmer,TN,KSZY,35.2,-88.5,186,99999


In [7]:
st_list = st_list[1281:]
st_list[0]

'KSZY'

In [ ]:

for sta in st_list:
    print('cleaning ' + sta)
    dat = pd.read_sql_query("SELECT * FROM wea WHERE st_code=? ",con,params=[sta])
    #dat.head()
    dat = clean_data(dat)
    #dat.to_csv('cleaned/' + sta + '_cleaned.csv',index=False)
    
    # write to S3
    s3 = boto3.resource('s3')
    #fname = csv_name
    key_name = 'cleaned/' + sta + '_cleaned.csv'
    #data = open(fname, 'rb')
    data = dat.to_csv(None,index=False)
    s3.Bucket('wundergrounddaily').put_object(Key=key_name, Body=data)

    # write to 'clean' database table
    dat.to_sql("wea_clean",con,if_exists='append',index=False)


cleaning KSZY


In [ ]:
sta = st_list[0]
dat = pd.read_sql_query("SELECT * FROM wea WHERE st_code=? ",con,params=[sta])
dat.head()

In [ ]:
dat = clean_data(dat)

In [ ]:
dat.info()

In [ ]:
dat.precip_In.dtype=='float64'